In [ ]:
# Setting up a custom stylesheet in IJulia
file = open("style.css") # A .css file in the same folder as this notebook file
styl = readall(file) # Read the file
HTML("$styl") # Output as HTML

# Types

<h2>In this lesson</h2>

- [Introduction](#Introduction)
- [Importing the packages for this lesson](#Importing-the-packages-for-this-lesson)
- [Outcomes](#Outcomes)
- [The type system in Julia](#The-type-system-in-Julia)
- [Type creation](#Type-creation)
- [Conversion and promotion](#Convertion-and-promotion)
- [Parametrizing a type](#Parametrizing-a-type)
- [The equality of values](#The-equality-of-values)
- [Defining methods for functions that will use user-types](#Defining-methods-for-functions-that-will-use-user-types)
- [Constraining field values](#Constraining-field-values)
- [More complex parameters](#More-complex-parameters)
- [Screen output of a user-defined type](#Screen-output-of-a-user-defined-type)

<hr>
<h2>Introduction</h2>

A computer variable, which is a space in memory, holds values of different types, i.e. integers, floating point values, and strings.  In some languages the type of the value to be held inside of a variable must be explicitely declared.  These languages are termed *statically typed*.  In *dynamically typed* languages nothing is known about the type of the value held inside the variable until runtime.  Being able to write code operating on different types is termed *polymorphism*.

Julia is a dynamically typed language, yet it is possible to declare a type for values as well.  Declaring a type allows for code that is clear to understand.  As an assertion it can help to confirm that your code is working as expected.  It can also allow for faster code execution by providing the compiler with extra information.

[Back to the top](#In-this-lesson)

<hr>
<h2>Outcomes</h2>

After successfully completing this lecture, you will be able to:

- Understand the Julia type system
- Create your own user-defined types
- Parametrize your types
- Overload methods for Julia functions so that they can use your types

[Back to the top](#In-this-lesson)

<hr>
<h2>The type system in Julia</h2>

Julia holds a type hierarchy that flows like the branches of a tree.  Right at the top we have a type called `Any`.  All types are subtypes of this type.  Right at the final tip of the branches we have concrete types.  They can hold values.  Supertypes of these concrete types are called abtsract types and they cannot hold values, i.e. we cannot create an instance of an abstract type.

We can use Julia to see if types are subtypes of a supertype.

In [ ]:
# Is Number a subtype of Any?
Number <: Any

In [ ]:
# Is Float 64 a subtype of AbstractFloat?
Float64 <: AbstractFloat

In [ ]:
# The subtypes of Any
subtypes(Any)

In [ ]:
# Subtypes of AbstractString
subtypes(AbstractString)

In [ ]:
# Subtypes of Number
subtypes(Number)

In [ ]:
# Subtypes of Real
subtypes(Real)

In [ ]:
# Subtypes of AbstractFloat
subtypes(AbstractFloat)

In [ ]:
# Subtypes of Integer
subtypes(Integer)

In [ ]:
# Subtypes of Signed
subtypes(Signed)

[Back to the top](#In-this-lesson)

<hr>
<h2>Declaring a type</h2>

A type is declared by the double colon, `::`, sign.  To the left we place the value (or placeholder for a variable, i.e. a variable name) and to the right the actual type.  In the example below we want to express the fact that the value $ 2 + 2 $ is an instance of a 64-bit integer.

In [ ]:
(2 + 2)::Int64

If we typed `(2 + 2)::Float64` we would get the following error:
```
LoadError: TypeError: typeassert: expected Float64, got Int64
while loading In[3], in expression starting on line 1
```
We used the declaration of a type as an assertion, which allowed us to see that there was something wrong with our code.  We can imagine a program where the `+()` (or a more complicated user-defined) function is called and we need the arguments to be of a certain type.  An error such as the one above can give us information about what went wrong.

Declaring a type of a local variable (inside of a function), we state that the type should always remain the same.  This is more like what would happen in a statically typed language.  It is really helpful if we want an error to be thrown should the type of a variable be changed by another part of our code.  This can lead to type instability.  It can really impact the speed of execution.

In [ ]:
# Creating a function with a local variable
function static_local_variable()
    v::Int16 = 42
    return v
end

In [ ]:
# Calling the function
static_local_variable()

In [ ]:
# Checking the type of the answer just give
typeof(ans)

Remember that `v` is local to the function.  If we try and look at it value by typing `v`, we would get the following error:
```
LoadError: UndefVarError: v not defined
while loading In[7], in expression starting on line 1
```

Now that we know something about declaring a type, let's look at creating our own types.

[Back to the top](#In-this-lesson)

<hr>
<h2>Type creation</h2>

As mentioned, we can create our own types.  Consider a Cartesian coordinate system along two perpendicular axes, say $ x $ and $ y $.  A vector in the plane can be represented as a type.  The keyword we use to create a type is `type`.  If we want instances of our type to be immutable, we use the keyword `immutable`.

In [1]:
# Creating a concrete type called Vector_2D
mutable struct Vector_2D
    x::Float64 # x is a fieldname of the type and has an optional type
    y::Float64 # y is a fieldname of the type and has an optional type
end

fieldnames(Vector_2D)
vector2d = Vector_2D(0.1,0.1)
typeof(vector2d)

Vector_2D

This is actually a composite type, since we have fields.  Composite objects declared with struct are immutable; they cannot be modified after construction. This may seem odd at first, but it has several advantages:

- It can be more efficient. Some structs can be packed efficiently into arrays, and in some cases the compiler is able to avoid allocating immutable objects entirely.
- It is not possible to violate the invariants provided by the type's constructors.
- Code using immutable objects can be easier to reason about.
- An immutable object might contain mutable objects, such as arrays, as fields. Those contained objects will remain mutable; only the fields of the immutable object itself cannot be changed to point to different objects.

For a type that is non-composite we can imagine a simple wrapper around an already defined type, such as we do below. 

In [6]:
# A non-composite type
struct NonComposite
    x::Float64
end

In [7]:
my_non_composite = NonComposite(42)

NonComposite(42.0)

In [8]:
# Type
typeof(ans)

NonComposite

Back to the more exciting composite types.  We can now instantiate the concrete type `Vector_2D`.

In [9]:
vector_1 = Vector_2D(2, 2)

Vector_2D(2.0, 2.0)

In [10]:
# The type of vector_1
typeof(vector_1)

Vector_2D

Notice how we get floating point values even though we gave two integer values.  The `convert()` functions was created to change allowable values to 64-bit floating point values.

Also notice that it looks like we called a function when we typed `Vector_2D(2, 2)`.  When we define a type, constructors are created.  They allow us to create an instance of that type (sometimes referred to as an *object* of that type).

As with functions, we can access the methods that were created with the type.

In [11]:
methods(Vector_2D)

# 2 methods for generic function "(::Type)":
[1] Vector_2D(x::Float64, y::Float64) in Main at In[5]:3
[2] Vector_2D(x, y) in Main at In[5]:3

We can also access the fieldnames and their values.  They are mutable, i.e. we can pass new values to them.

In [12]:
# The available names (fields, fieldnames)
# Note that they are of type Symbol
fieldnames(Vector_2D)

(:x, :y)

In [13]:
# Getting the value of the :x field
vector_1.x

2.0

In [14]:
# Alternative syntax using the field's symbol representation
getfield(vector_1, :x)

2.0

In [15]:
# Another alternative notation using the index number of the fields
getfield(vector_1, 1)

2.0

In [16]:
vector_1.x = 3

3

In [17]:
vector_1

Vector_2D(3.0, 2.0)

Another way to pass a value to a fieldname in a type is the `setfield()` function.  We have to use the correct type for the value.  If we use an integer such as `setfield!(vector_1, :x, 4)` we would get the follwoing error:
```
LoadError: TypeError: setfield!: expected Float64, got Int64
while loading In[25], in expression starting on line 1
```

In [18]:
# Now we have to use a floating point value
setfield!(vector_1, :x, 4.0)

4.0

In [19]:
# vector_1 has been changed
vector_1

Vector_2D(4.0, 2.0)

[Back to the top](#In-this-lesson)

<hr>
<h2>Convertion and promotion</h2>

Before we go any further, we must have a look behind the scenes.  Above we saw that an integer was converted to a floating point value as specified for the fields of our new type.

In [20]:
# Using the convert function
convert(Float64, 10)

10.0

If precision is lost, convertion will result in an error.  For instance, `convert(Int16, 10.1)` will return:
```
adError: InexactError()
while loading In[20], in expression starting on line 1
```
Using `convert(Int16, 10.0)` will return a value of $ 10 $, though.

Julia has a type promotion system that will try to incorporate values into a single type.  If we pass an integer and a floating point value, the integer will be promoted to a floating point value.

In [21]:
promote(10, 10.0)

(10.0, 10.0)

In [22]:
typeof(10)

Int64

In [23]:
typeof(10.0)

Float64

This promotion to a common type lifts the lid on multiple dispatch when a function is called with unspecified argument types (i.e. `Any`).

[Back to the top](#In-this-lesson)

<hr>
<h2>Parametrizing a type</h2>

When creating a user type, we need not specify the type explicitely.  We could use a parameter.  Have a look at the example below.

In [ ]:
type Vector_3D{T}
    x::T
    y::T
    z::T
end

We use $ T $ as a parameter placeholder.  When we instantiate the type we can use any appropriate type, as long as all the fields values are of the same type.

In [ ]:
# Using 64 bit integers
vector_2 = Vector_3D(10, 12, 8)

If we were to execute `vector_2 = Vector_3D(10.1, 10, 8)`, we would get the following error:
```
LoadError: MethodError: `convert` has no method matching convert(::Type{Vector_3D{T}}, ::Float64, ::Int64, ::Int64)
This may have arisen from a call to the constructor Vector_3D{T}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  Vector_3D{T}(::T, !Matched::T, !Matched::T)
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
while loading In[28], in expression starting on line 1
```

We can constrain the parametric type.  Below we allow all subtypes of of the abstract type `Real`.

In [ ]:
type Vector_3D_Real{T <: Real}
    x::T
    y::T
    z::T
end

As an aside, we cannot redefine a type.  If we would use:
```
type Vector_3D{T}
    x::T
    y::T
    z::T
end
```
we would get the error:
```
LoadError: invalid redefinition of constant Vector_3D
while loading In[98], in expression starting on line 1
```

In [ ]:
# Creating a new instance
vector_3 = Vector_3D_Real(3, 3, 3)

[Back to the top](#In-this-lesson)

<hr>
<h2>The equality of values</h2>

When are two values equal?  We use a double equal sign to return a Boolean value.

In [ ]:
# Using the functional notation
==(5, 5.0)

Numbers are immutable and are compared at the bit level.  This includes their types.  We can use the `===` sign or the `is()` function to check for equality.

In [ ]:
is(5, 5.0)

Where does this leave our user-defined types?  We will see below that the address in memory is checked when dealing with more complex objects such as our user-defined, composite types.

In [ ]:
vector_a = Vector_2D(1.0, 1.0)
vector_b = Vector_2D(1.0, 1.0)

In [ ]:
is(vector_a, vector_b)

[Back to the top](#In-this-lesson)

<hr>
<h2>Defining methods for functions that will use user-types</h2>

The summation function, `+()` has methods for adding different types.  What if we want to add two instances of our `Vector_2D` user-type?  If we were to add `vector_a` to `vector_b` we would get the following error:
```
LoadError: MethodError: `+` has no method matching +(::Vector_2D, ::Vector_2D)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
```

In [ ]:
# Base methods for the +() function
methods(+)

We have to create a method.

In [ ]:
import Base.+

In [ ]:
+(u::Vector_2D, v::Vector_2D) = Vector_2D(u.x + v.x, u.y + v.y)

In [ ]:
+(vector_a, vector_b)

[Back to the top](#In-this-lesson)

<hr>
<h2>Constraining field values</h2>

We can well imagine needing to constain the values that a type can hold.  Below we create the Bloodpressure type with two fields that hold integer values.  They cannot be negative and the systolic blood pressure must be higher than the diastolic blood pressure.  We solve this problem by creating an inner constructor.

In [ ]:
type BloodPressure
    # Don't leave as Any
    systolic::Int16
    diastolic::Int16
    function BloodPressure(s, d)
        # Using short-circuit evaluations && and ||
        s < 0 && throw(ArgumentError("Negative pressures are not allowed!"))
        s <= d && throw(ArgumentError("The systolic blood pressure must be higher than the diastolic blood pressure!"))
        isa(s, Integer) || throw(ArgumentError("Only integer values allowed!"))
        isa(d, Integer) || throw(ArgumentError("Only integer values allowed!"))
        new(s, d)
    end
end

In [ ]:
bp_1 = BloodPressure(120, 80)

Using `bp_2 = BloodPressure(-1, 90)` will result in the error:
```
LoadError: ArgumentError: Negative pressures are not allowed!
while loading In[32], in expression starting on line 1
```

Using `bp_2 = BloodPressure(80, 120)` will result in the error:
```
LoadError: ArgumentError: The systolic blood pressure must be higher than the diastolic blood pressure
while loading In[56], in expression starting on line 1
```

Using `bp_2 = BloodPressure(120.0, 80)` will result in the error:
```
LoadError: ArgumentError: Only integer values allowed!
while loading In[95], in expression starting on line 1
```

Beware.  Using inner constructors with parametrized types can lead to problems.

In [ ]:
type BloodPressureParametrized{T <: Real}
    # Don't leave as Any
    systolic::T
    diastolic::T
    function BloodPressureParametrized(s, d)
        s < 0 && throw(ArgumentError("Negative pressures are not allowed!"))
        s <= d && throw(ArgumentError("The systolic blood pressure must be higher than the diastolic blood pressure!"))
        isa(s, Integer) || throw(ArgumentError("Only integer values allowed!"))
        isa(d, Integer) || throw(ArgumentError("Only integer values allowed!"))
        new(s, d)
    end
end

Using `bp_3 = BloodPressureParametrized(120, 80)` will result in the error;
```
LoadError: MethodError: `convert` has no method matching convert(::Type{BloodPressureParametrized{T<:Real}}, ::Int64, ::Int64)
This may have arisen from a call to the constructor BloodPressureParametrized{T<:Real}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
while loading In[102], in expression starting on line 1

 in call at essentials.jl:57
 ```

Now we have to specify the type during the instantiation.

In [ ]:
bp_3 = BloodPressureParametrized{Int}(120, 80)

In [ ]:
type BloodPressureParametrizedFixed{T <: Real}
    systolic::T
    diastolic::T
    function BloodPressureParametrizedFixed(s, d)
        s < 0 && throw(ArgumentError("Negative pressures are not allowed!"))
        s <= d && throw(ArgumentError("The systolic blood pressure must be higher than the diastolic blood pressure!"))
        new(s, d)
    end
end

We can fix this by the assignment below.

In [ ]:
# A bit of an effort
BloodPressureParametrizedFixed{T}(systolic::T, diastolic::T) = BloodPressureParametrizedFixed{T}(systolic, diastolic)

In [ ]:
bp_4 = BloodPressureParametrizedFixed(120, 80)

We can get way more specific.  In the code below we tell Julia that if we pass integers to the type, they should be expressed as floating point values.

In [ ]:
BloodPressureParametrizedFixed{T <: Int}(systolic::T, diastolic::T) = BloodPressureParametrizedFixed{Float64}(systolic, diastolic)

In [ ]:
bp_5 = BloodPressureParametrizedFixed(120, 80)

[Back to the top](#In-this-lesson)

<hr>
<h2>More complex parameters</h2>

Up until now we have constrained ourselves to a single parameter.  It is possible, though, to create more than one.  Below we create a type called `Relook`.  It has one fieldname called `duration`, which must be of subtype, `Real`.  There is also a second parameter.

In [ ]:
type Relook{N, T<:Real}
    duration::T
end

Using `patient_1 = Relook(3, 60)` will result in the error:
```
LoadError: MethodError: `convert` has no method matching convert(::Type{Relook{N,T}}, ::Int64, ::Int64)
This may have arisen from a call to the constructor Relook{N,T}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
while loading In[175], in expression starting on line 1

 in call at essentials.jl:57
```

In [ ]:
# We have to specify the type of the second parameter
patient_1 = Relook{4, Int16}(60)

In [ ]:
patient_1.duration

We now want to add only objects (instances) with the same value in the first parameter.

In [ ]:
+{N, T}(u::Relook{N, T}, v::Relook{N, T}) = Relook{N, T}(u.duration + v.duration)

In [ ]:
patient_2 = Relook{4, Int16}(70)

In [ ]:
patient_1 + patient_2

In [ ]:
patient_3 = Relook{3, Int16}(70)

Using `patient_1 + patient_3` will now result in the error:
```LoadError: MethodError: `+` has no method matching +(::Relook{4,Int16}, ::Relook{3,Int16})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +{N,T}(::Relook{N,T}, !Matched::Relook{N,T})
while loading In[191], in expression starting on line 1
```

In [ ]:
patient_4 = Relook{4.0, Int16}(70)

Using `patient_1 + patient_4` will also result in an error, because the types of `N` do not match.  The error would be:
```
LoadError: MethodError: `+` has no method matching +(::Relook{4,Int16}, ::Relook{4.0,Int16})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +{N,T}(::Relook{N,T}, !Matched::Relook{N,T})
  +{N,T1,T2}(::Relook{N,T1}, !Matched::Relook{N,T2})
while loading In[210], in expression starting on line 1
```

Below we fix the fieldname type mismatch.

In [ ]:
+{N, T1, T2}(u::Relook{N, T1}, v::Relook{N, T2}) = Relook{N, promote_type(T1, T2)}(u.duration + v.duration)

In [ ]:
patient_5 = Relook{4, Float64}(60)

In [ ]:
# N = 4 for both patient_1 and patient_5
# T for patient_1 is Int16 and T for patient_2 is Float64
patient_1 + patient_5

If we want to throw an error if the number of relooks are not equal when trying to add to obejcts of the `Relook` type, we can do the following.

In [ ]:
+{N1, N2, T}(u::Relook{N1, T}, v::Relook{N2, T}) = 
throw(ArgumentError("Cannot add durations when the number of relooks do not match."))

Using `patient_1 + patient_3` will now result in the error:
```
LoadError: ArgumentError: Cannot add durations when the number of relooks do not match.
while loading In[237], in expression starting on line 1

 in + at In[236]:1
```

What about calculating the natural logarithm of the duration of a `Relook` object?  We could just specify the field name.

In [ ]:
log(patient_1.duration)

Better still, we could specify wat the `log` function actually does with a `Relook` object.

In [ ]:
import Base.log

In [ ]:
log(u::Relook) = log(u.duration)

In [ ]:
log(patient_1)

We can specify a `convert` method that will convert all our `Relook` objects to numerical values which we can pass to Julia functions.

In [ ]:
import Base.convert

In [ ]:
# The float() function tries to convert a value to a floating point value
convert(::Type{AbstractFloat}, u::Relook) = float(u.duration)

In [ ]:
# Covreting our Relook object and passing it to log10()
log10(convert(AbstractFloat, patient_1))

[Back to the top](#In-this-lesson)

<hr>
<h2>Screen output of a user-defined type</h2>

We can create some meaning to our types by the way an object of the type is represented on the screen.  Above we just saw two values we instantiation our type `Relook`.  Let's change that a bit by overloading the `show` function.

In [ ]:
import Base.show

In [ ]:
show{N, T}(io::IO, u::Relook{N, T}) = print(io, "Patient with ", N, " relook procedures totalling ", u.duration,
" minutes.")

In [ ]:
patient_6 = Relook{4, Int16}(60)

[Back to the top](#In-this-lesson)